In [1]:
from PyQt5.QtWidgets import*
import sys
import cv2 as cv
import numpy as np

class Video(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('비디오에서 프레임 수집')
        self.setGeometry(200,200,500,100)
        
        videoButton=QPushButton('비디오 켜기',self)
        captureButton=QPushButton('프레임 잡기',self)
        saveButton=QPushButton('프레임 저장',self)
        quitButton=QPushButton('나가기',self)
        collectButton=QPushButton('여러 프레임 잡기',self) # 콜렉트 버튼과 쇼버튼 추가
        showButton=QPushButton('영상 보기',self)
        
        videoButton.setGeometry(10,10,100,30)  # setGeometry(x, y, width, height)
        captureButton.setGeometry(110,10,100,30)
        saveButton.setGeometry(210,10,100,30)
        quitButton.setGeometry(310,10,100,30)
        collectButton.setGeometry(10,40,130,30) # 콜렉트 버튼과 쇼버튼 좌표 설정
        showButton.setGeometry(140,40,100,30)
        
        videoButton.clicked.connect(self.videoFunction)
        captureButton.clicked.connect(self.captureFunction)
        saveButton.clicked.connect(self.saveFunction)
        quitButton.clicked.connect(self.quitFunction)
        collectButton.clicked.connect(self.collectFunction) # 콜렉트 버튼은 클릭시 함수 콜렉트펀션 연결하고 쇼버튼도 마찬가지로 함수 쇼펀션 연결
        showButton.clicked.connect(self.showFunction)

    def showFunction(self):             # 쇼 펀션 함수 추가
        stack=cv.resize(self.imgs[0],dsize=(0,0),fx=0.25,fy=0.25)
        for i in range(1,len(self.imgs)):
            stack=np.hstack((stack,cv.resize(self.imgs[i],dsize=(0,0),fx=0.25,
            fy=0.25)))
        cv.imshow('Image collection',stack)
        
    def videoFunction(self):
        self.cap=cv.VideoCapture(0,cv.CAP_DSHOW)
        if not self.cap.isOpened():self.close()

        while True:
            ret,self.frame=self.cap.read()
            if not ret:break
            cv.imshow('video display',self.frame)
            cv.waitKey(1)

    def captureFunction(self):
        self.capturedFrame=self.frame
        cv.imshow('Captured Frame',self.capturedFrame)

    def saveFunction(self):
        fname=QFileDialog.getSaveFileName(self,'파일저장','/')
        cv.imwrite(fname[0],self.capturedFrame)

    def quitFunction(self):
        self.cap.release()
        cv.destroyAllWindows()
        self.close()

    def collectFunction(self):      # 콜렉트 펀션 함수 추가
        self.cap=cv.VideoCapture(0,cv.CAP_DSHOW)
        if not self.cap.isOpened(): sys.exit('카메라 연결 실패')
            
        self.imgs=[]
        while True:
            ret,frame=self.cap.read()
            if not ret: break
                
            cv.imshow('video display',frame)
            
            key=cv.waitKey(1)
            if key==ord('c'):
                self.imgs.append(frame)
            elif key==ord('q'):
                self.cap.release()
                cv.destroyWindow('video display')
                break
                
    
app=QApplication(sys.argv)
win=Video()
win.show()
app.exec_()

0